# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [35]:
# Import libraries
import numpy as np
import pandas as pd

In [36]:
# Importing the dataset
dataset = pd.read_csv('./data/pokemon.csv') 

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [37]:
    dataset['Calculated_Total'] = (dataset['HP'] + dataset['Attack'] + dataset['Defense'] +dataset['Sp. Atk'] + dataset['Sp. Def'] + dataset['Speed'])
    print(dataset['Calculated_Total'])
    #check our calculated total maches the actual total field
    dataset['Total_Match'] =  dataset['Total'] == dataset['Calculated_Total']
    match_count = dataset['Total_Match'].sum()
    total_dataset = len(dataset)

0      318
1      405
2      525
3      625
4      309
      ... 
795    600
796    700
797    600
798    680
799    600
Name: Calculated_Total, Length: 800, dtype: int64


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](https://education-team-2020.s3.eu-west-1.amazonaws.com/ds-ai/lab-dataframe-calculation-and-transformation/images/one-hot-encoding.png)

In [39]:
dataset['Type 2'] = dataset['Type 2'].fillna('None') # handel the NaN values in Type 2
type1_encoded = pd.get_dummies(dataset['Type 1'], prefix='Type') # On e Hot Encoding on Type 1
type2_encoded = pd.get_dummies(dataset['Type 2'], prefix='Type') # On e Hot Encoding on Type 2
all_types_encoded = type1_encoded.add(type2_encoded, fill_value=0)
all_types_encoded = all_types_encoded.clip(upper=1)
dataset_encoded = pd.concat([dataset, all_types_encoded], axis=1) #add encoded column
print(f" Original dataframe={dataset.shape}")
print(f"Encoded dataframe={dataset_encoded.shape}")
print(f" New type column created={list(all_types_encoded.columns)}")




 Original dataframe=(800, 15)
Encoded dataframe=(800, 34)
 New type column created=['Type_Bug', 'Type_Dark', 'Type_Dragon', 'Type_Electric', 'Type_Fairy', 'Type_Fighting', 'Type_Fire', 'Type_Flying', 'Type_Ghost', 'Type_Grass', 'Type_Ground', 'Type_Ice', 'Type_None', 'Type_Normal', 'Type_Poison', 'Type_Psychic', 'Type_Rock', 'Type_Steel', 'Type_Water']


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [52]:
type_columns =[col for col in dataset_encoded.columns if col.startswith('Type_')]
correlations ={}
for type_col in type_columns:
    correlation = dataset_encoded['Total'].corr(dataset_encoded[type_col])
    type_name = type_col.replace('Type_', '')    
    correlations[type_name] = correlation
sorted_correlations = sorted(correlations.items(), key=lambda x: x[1], reverse=True) # sort by correlation streght
print('sorted_correlations', sorted_correlations)

sorted_correlations [('Dragon', np.float64(0.2297047163217014)), ('Psychic', np.float64(0.12468802510752963)), ('Steel', np.float64(0.10970325430519122)), ('Fire', np.float64(0.07872605613705123)), ('Fighting', np.float64(0.07778621928342079)), ('Ice', np.float64(0.060248167691803606)), ('Flying', np.float64(0.05938340252570127)), ('Dark', np.float64(0.056153774076480284)), ('Rock', np.float64(0.032731200969134455)), ('Electric', np.float64(0.020971330428545147)), ('Ground', np.float64(0.015060471925699967)), ('Ghost', np.float64(0.0036405748393128533)), ('Water', np.float64(-0.021664867287180672)), ('Fairy', np.float64(-0.03669774114188013)), ('Grass', np.float64(-0.052591804857805895)), ('Poison', np.float64(-0.09044141387432197)), ('Normal', np.float64(-0.10533095102451885)), ('Bug', np.float64(-0.145781361252384)), ('None', np.float64(-0.1859449115474653))]


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here